<h1>Plain Fin Heat Transfer Coefficient and Pressure Drop</h1>

Let us take look at the calculations based on Wang et al. (2000) method to calculate heat transfer coefficient for a plain fin used with round tube plain fin heat exchangers.

The inputs required for this calculation are given below:
$\\$
$D_o$ = Tube Outside Diameter $[m]$

$F_p$ = Fin Pitch $[m]$

$P_t$ = Transverse tube pitch or vertical tube spacing $[m]$

$P_l$ = Longitudinal tube pitch or horizontal tube spacing $[m]$

$\delta_f$ = Fin Thermal Conductivity $[Wm^{-1}K^{-1}]$

$\delta_w$ = Tube Thermal Conductivity $[Wm^{-1}K^{-1}]$

$X_f$ = Half wavelength of the fin wave $[m]$

$P_d$ = Pattern Depth or twice amplitude of fin wave $[m]$

In [1]:
import numpy as np
import HT_Library as ht

In [2]:
Do = 9.525e-3
Fp = 1.22e-3
Pl = 19.05e-3
Pt = 25.4e-3
deltaf = 0.152e-3
Pd = 1.23e-3
Xf = 7e-3

Based on these inputs we can calculate the following geometry parameters.
$\\$
$D_c$ = Fin Collar Outside Diameter $[m]$

In [3]:
Dc = 2*deltaf + Do
print('Dc =',Dc)

Dc = 0.009829000000000001


<h2>Minimum free flow area calculation</h2>
Reynolds area is based on maximum flow velocity. For this purpose, it is necessary to calculate minimum free flow area.
$\\$
$L_t$ = Length between headers $[m]$
$\\$
$A_o$ = Minimum free flow area $[m^2]$
$\\$
$N_{row}$ = Number of tube rows $[-]$
$\\$
$N_{tube/row}$ = Number of tubes per row $[-]$
$\\$
$FPM$ = Number of fins per unit length $[m^{-1}]$
$\\$
$A_{oi}$ = Minimum free flow area for inline tube configuration $[m^{2}]$
$\\$
$A_{fr}$ = Frontal area $[m^{2}]$

In [4]:
Lt = 0.5
L1 = Lt
print('L1 =',L1)
NtubesPerRow = 12
Nrow = 4
L2 = Nrow*Pl
print('L2 =',L2)
L3 = NtubesPerRow*Pt
print('L3 =',L3)
#FPM = 1/(Fp+deltaf)
FPM = 1/Fp
print('FPM =',FPM)
Fs = Fp-deltaf
print('Fs =',Fs)
Aoi = NtubesPerRow*((Pt-Do)*L1-(Pt-Do)*deltaf*FPM*L1)
print('Aoi =',Aoi)
Afr = L1*L3
print('Afr =',Afr)

L1 = 0.5
L2 = 0.0762
L3 = 0.30479999999999996
FPM = 819.672131147541
Fs = 0.001068
Aoi = 0.08338278688524589
Afr = 0.15239999999999998


In [5]:
a = 0.5*((Pt-Do)-(Pt-Do)*deltaf*FPM)
print('a =',a)
b = np.sqrt((Pt/2)**2+Pl**2)-Do-(Pt-Do)*deltaf*FPM
print('b =',b)
c = 2*min(a,b)
print('c =',c)

Aos = L1*((NtubesPerRow-1)*c+(Pt-Do)-(Pt-Do)*deltaf*FPM)
print('Aos =',Aos)

a = 0.0069485655737704915
b = 0.011392381746737314
c = 0.013897131147540983
Aos = 0.0833827868852459


In [6]:
sigma = (Aos)/(L3*L1)
print('sigma =',sigma)

sigma = 0.5471311475409837


In [7]:
u = 4
u_max = u/sigma
print('u_max =',u_max)
rho = 1.111
mu = 1.913e-5

Re_Dc = rho*u_max*Dc/mu
print('Re_Dc =',Re_Dc)

u_max = 7.310861423220973
Re_Dc = 4173.274733295351


In [8]:
theta_rad = np.arccos(Xf/(np.sqrt(Xf**2+Pd**2)))
theta_deg = theta_rad*180/np.pi
print('theta_deg =',theta_deg)

alpha = 1/np.cos(theta_rad)
print('alpha =',alpha)

theta_deg = 9.965950139437554
alpha = 1.01532039780755


In [9]:
Ap = np.pi*Do*(L1-deltaf*FPM*L1)*Nrow*NtubesPerRow+2*(L2*L3-np.pi*Do*Do/4*Nrow*NtubesPerRow)
print('Ap =',Ap)

As = 2*(L2*L3*alpha-np.pi/4*Do**2*Nrow*NtubesPerRow)*FPM*L1+2*L3*alpha*deltaf*FPM*L1
print('As =',As)

Dh = 4*Aos*L2/(Ap+As)
print('Dh =',Dh)

Ap = 0.6683023707470602
As = 16.564222775882655
Dh = 0.0014748316469216675


In [10]:
class WavyHerringboneFinHX:
    def __init__(self,tubeArrangement,NtubesPerRow,Nrow,Lt,Do,Fp,Pl,Pt,deltaf,Pd,Xf):
        self.NtubesPerRow = NtubesPerRow
        self.Nrow = Nrow
        self.Do = Do
        self.Fp = Fp
        self.Pl = Pl
        self.Pt = Pt
        self.deltaf = deltaf        
        self.Lt = Lt
        self.Pd = Pd
        self.Xf = Xf
        
        self.Dc = 2*deltaf + Do
        self.L1 = Lt
        self.L2 = Nrow*Pl
        self.L3 = NtubesPerRow*Pt
        self.FPM = 1/(Fp)
        self.Fs = Fp-deltaf
        self.a = 0.5*((Pt-Do)-(Pt-Do)*deltaf*self.FPM)
        self.b = np.sqrt((Pt/2)**2+Pl**2)-Do-(Pt-Do)*deltaf*self.FPM
        self.c = 2*min(self.a,self.b)
        self.theta_rad = np.arccos(self.Xf/(np.sqrt(self.Xf**2+self.Pd**2)))
        self.alpha = 1/np.cos(self.theta_rad)
        
        if(tubeArrangement.lower()=='inline'):
            self.Ao = NtubesPerRow*((Pt-Do)*self.L1-(Pt-Do)*deltaf*self.FPM*self.L1)
        else:
            self.Ao = self.L1*((NtubesPerRow-1)*self.c+(Pt-Do)-(Pt-Do)*deltaf*self.FPM)
            
        self.Ap = (np.pi*Do*(self.L1-deltaf*self.FPM*self.L1)*Nrow
                   *NtubesPerRow+2*(self.L2*self.L3-np.pi*Do*Do/4*Nrow*NtubesPerRow))
        self.As = (2*(self.L2*self.L3*self.alpha-np.pi/4*Do**2*Nrow*NtubesPerRow)
                   *self.FPM*self.L1+2*self.L3*self.alpha*deltaf*self.FPM*self.L1)
        self.Afr = self.L1*self.L3
        self.sigma = self.Ao/self.Afr
        self.At = (self.Ap+self.As)
        self.Dh = 4*self.Ao*self.L2/self.At

In [11]:
x = WavyHerringboneFinHX('staggered',NtubesPerRow,Nrow,Lt,Do,Fp,Pl,Pt,deltaf,Pd,Xf)

In [12]:
print('Ap =',x.Ap)
print('As =',x.As)
print('sigma =',x.sigma)
print('Dh =',x.Dh)
print('Fp =',x.Fp)
print('Fs =',x.Fs)

Ap = 0.6683023707470602
As = 16.564222775882655
sigma = 0.5471311475409837
Dh = 0.0014748316469216675
Fp = 0.00122
Fs = 0.001068


<h2>HTC Calculations</h2>

In [13]:
J1 = (0.0045-0.491
      *(np.power(Re_Dc,-0.0316-0.0171*np.log(x.Nrow*np.tan(x.theta_rad))))
      *(np.power(x.Pl/x.Pt,-0.109*np.log(x.Nrow*np.tan(x.theta_rad))))
      *(np.power(x.Dc/x.Dh,0.542+0.0471*x.Nrow))
      *(np.power(x.Fs/x.Dc,0.984))
      *np.power(x.Fs/x.Pt,-0.349))
print('J1 =',J1)

J2 = -2.72+6.84*np.tan(x.theta_rad)
print('J2 =',J2)

J3 = 2.66*np.tan(x.theta_rad)
print('J3 =',J3)


j = (0.882*np.power(Re_Dc,J1)*np.power(x.Dc/x.Dh,J2)*np.power(x.Fs/x.Pt,J3)
     *np.power(x.Fs/x.Dc,-1.58)*np.power(np.tan(x.theta_rad),-0.2))
print('j =',j)

J1 = -0.5290585204550293
J2 = -1.5181142857142862
J3 = 0.4673999999999999
j = 0.006460227229246512


In [14]:
c1 = np.power(Re_Dc,J1)
print('c1 =',c1)
c2 = np.power(x.Dc/x.Dh,J2)
print('c2 =',c2)
c3 = np.power(x.Fs/x.Pt,J3)
print('c3 =',c3)
c4 = np.power(x.Fs/x.Dc,-1.58)
print('c4 =',c4)
c5 = np.power(np.tan(x.theta_rad),-0.2)
print('c5 =',c5)

c1 = 0.012149427331634143
c2 = 0.056160080394714734
c3 = 0.2273709227781303
c4 = 33.34434341704458
c5 = 1.4159195744224213


In [15]:
j1 = (-0.0545-0.0538*np.tan(x.theta_rad)-0.302*np.power(x.Nrow,-0.24)*np.power(x.Fs/x.Pl,-1.3)
      *np.power(x.Pl/x.Pt,0.379)*np.power(x.Pl/x.Dh,-1.35)*np.power(np.tan(x.theta_rad),-0.256))
print('j1 =',j1)

j2 = (-1.29*np.power(x.Pl/x.Pt,1.77-9.43*np.tan(x.theta_rad))
      *np.power(x.Dc/x.Dh,0.229-1.43*np.tan(x.theta_rad))
      *np.power(x.Nrow,-0.166-1.08*np.tan(x.theta_rad))
      *np.power(x.Fs/x.Pt,-0.174*np.log(0.5*Nrow)))
print('j2 =',j2)

j = (0.0646*np.power(Re_Dc,j1)*np.power(x.Dc/x.Dh,j2)*np.power(x.Fs/x.Pt,-1.03)
     *np.power(x.Pl/x.Dc,0.432)*np.power(np.tan(x.theta_rad),-0.692)*np.power(Nrow,-0.737))
print('j =',j)

j1 = -0.46960679182139253
j2 = -1.0713105095162012
j = 0.00704852178730763


In [16]:
cp = 1049
Pr = 0.7336

ho = j*rho*u_max*cp/(np.power(Pr,(2/3)))
print('ho =',ho)

ho = 73.83290060732332


In [17]:
Temp = 313.75
Rh = 0.5
P_amb = 101325
y = ht.AirThermoPhysicalProps(P_amb,Temp,Rh)
print('rho =',y.rho)
print('mu =',y.mu)
print('cp =',y.cp)
print('Pr =',y.Pr)

rho = 1.109356342851275
mu = 1.8946030732764537e-05
cp = 1028.1616087016457
Pr = 0.7131921984075685


<h2>Pressure Drop Calculations</h2>

In [18]:
F1 = (-0.574-0.137*np.power(np.log(Re_Dc)-5.26,0.245)
      *np.power(x.Pt/x.Dc,-0.765)
      *np.power(x.Dc/x.Dh,-0.243)
      *np.power(x.Fs/x.Dh,-0.474)
      *np.power(np.tan(x.theta_rad),-0.217)
      *np.power(Nrow,0.035))
print('F1 =',F1)

F2 = -3.05*np.tan(x.theta_rad)
print('F2 =',F2)

F3 = -0.192*Nrow
print('F3 =',F3)

F4 = -0.646*np.tan(x.theta_rad)
print('F4 =',F4)

f = 4.37*np.power(Re_Dc,F1)*np.power(x.Fs/x.Dh,F2)*np.power(x.Pl/x.Pt,F3)*np.power(x.Dc/x.Dh,0.2054)*np.power(Nrow,F4)
print('f =',f)

F1 = -0.6721927917873555
F2 = -0.5359285714285713
F3 = -0.768
F4 = -0.11351142857142855
f = 0.030113319917351344


In [19]:
f1 = (-0.141*np.power(x.Fs/x.Pl,0.0512)*np.power(np.tan(x.theta_rad),-0.472)
      *np.power(x.Pl/x.Pt,0.35)*np.power(x.Pt/x.Dh,0.449*np.tan(x.theta_rad))*np.power(Nrow,-0.049+0.237*np.tan(theta_rad)))
print('f1 =',f1)

f2 = -0.562*np.power(np.log(Re_Dc),-0.0923)*np.power(Nrow,0.013)
print('f2 =',f2)

f3 = 0.302*np.power(Re_Dc,0.03)*np.power(x.Pt/x.Dc,0.026)
print('f3 =',f3)

f4 = -0.306 + 3.63*np.tan(theta_rad)
print('f4 =',f4)

f = (0.228*np.power(Re_Dc,f1)*np.power(np.tan(theta_rad),f2)*np.power(x.Fs/x.Pl,f3)*np.power(x.Pl/x.Dc,f4)
     *np.power(x.Dc/x.Dh,0.383)*np.power(x.Pl/x.Pt,-0.247))
print('f =',f)

f1 = -0.3097187975996135
f2 = -0.470496616888362
f3 = 0.3975041674302362
f4 = 0.33184285714285705
f = 0.03437509064240163


In [20]:
G = y.rho*u_max
print('G =',G)

G = 8.110350491556886


In [21]:
dP = f*G**2/(2*y.rho)*(x.As+x.Ap)/x.Ao
print('dP =',dP)

dP = 210.61754472177074


In [28]:
def CCWangWavyHerringboneFinCalc(HX,AirProps,InletVelocity):
    u_max = InletVelocity/HX.sigma
    Re_Dc = AirProps.rho*u_max*HX.Dc/AirProps.mu 

    if(Re_Dc<1000):
        J1 = (0.0045-0.491
              *(np.power(Re_Dc,-0.0316-0.0171*np.log(HX.Nrow*np.tan(HX.theta_rad))))
              *(np.power(HX.Pl/HX.Pt,-0.109*np.log(HX.Nrow*np.tan(HX.theta_rad))))
              *(np.power(HX.Dc/HX.Dh,0.542+0.0471*HX.Nrow))
              *(np.power(HX.Fs/HX.Dc,0.984))
              *np.power(HX.Fs/HX.Pt,-0.349))
        J2 = -2.72+6.84*np.tan(HX.theta_rad)
        J3 = 2.66*np.tan(HX.theta_rad)
        j = (0.882*np.power(Re_Dc,J1)*np.power(HX.Dc/HX.Dh,J2)*np.power(HX.Fs/HX.Pt,J3)
             *np.power(HX.Fs/HX.Dc,-1.58)*np.power(np.tan(HX.theta_rad),-0.2))
        F1 = (-0.574-0.137*np.power(np.log(Re_Dc)-5.26,0.245)
              *np.power(HX.Pt/HX.Dc,-0.765)
              *np.power(HX.Dc/HX.Dh,-0.243)
              *np.power(HX.Fs/HX.Dh,-0.474)
              *np.power(np.tan(HX.theta_rad),-0.217)
              *np.power(HX.Nrow,0.035))
        F2 = -3.05*np.tan(HX.theta_rad)
        F3 = -0.192*HX.Nrow
        F4 = -0.646*np.tan(HX.theta_rad)
        f = (4.37*np.power(Re_Dc,F1)*np.power(HX.Fs/HX.Dh,F2)
             *np.power(HX.Pl/HX.Pt,F3)*np.power(HX.Dc/HX.Dh,0.2054)*np.power(HX.Nrow,F4))        
    else:
        j1 = (-0.0545-0.0538*np.tan(HX.theta_rad)-0.302*np.power(HX.Nrow,-0.24)*np.power(HX.Fs/HX.Pl,-1.3)
              *np.power(HX.Pl/HX.Pt,0.379)*np.power(HX.Pl/HX.Dh,-1.35)*np.power(np.tan(HX.theta_rad),-0.256))
        j2 = (-1.29*np.power(HX.Pl/HX.Pt,1.77-9.43*np.tan(HX.theta_rad))
              *np.power(HX.Dc/HX.Dh,0.229-1.43*np.tan(HX.theta_rad))
              *np.power(HX.Nrow,-0.166-1.08*np.tan(HX.theta_rad))
              *np.power(HX.Fs/HX.Pt,-0.174*np.log(0.5*HX.Nrow)))
        j = (0.0646*np.power(Re_Dc,j1)*np.power(HX.Dc/HX.Dh,j2)*np.power(HX.Fs/HX.Pt,-1.03)
             *np.power(HX.Pl/HX.Dc,0.432)*np.power(np.tan(HX.theta_rad),-0.692)*np.power(HX.Nrow,-0.737)) 
        f1 = (-0.141*np.power(HX.Fs/HX.Pl,0.0512)*np.power(np.tan(HX.theta_rad),-0.472)
              *np.power(HX.Pl/HX.Pt,0.35)*np.power(HX.Pt/HX.Dh,0.449*np.tan(HX.theta_rad))
              *np.power(HX.Nrow,-0.049+0.237*np.tan(HX.theta_rad)))
        f2 = -0.562*np.power(np.log(Re_Dc),-0.0923)*np.power(HX.Nrow,0.013)
        f3 = 0.302*np.power(Re_Dc,0.03)*np.power(HX.Pt/HX.Dc,0.026)
        f4 = -0.306 + 3.63*np.tan(HX.theta_rad)
        f = (0.228*np.power(Re_Dc,f1)*np.power(np.tan(HX.theta_rad),f2)*np.power(HX.Fs/HX.Pl,f3)*np.power(HX.Pl/HX.Dc,f4)
             *np.power(HX.Dc/HX.Dh,0.383)*np.power(HX.Pl/HX.Pt,-0.247))    
        
    G = AirProps.rho*u_max
    ho = j*G*AirProps.cp/(np.power(AirProps.Pr,(2/3)))
    dP = f*G**2/(2*AirProps.rho)*(HX.As+HX.Ap)/HX.Ao
    return ho,dP